## **Machine Learning Approach for Early Diagnosis of Type 2 Diabetes in the Spanish Adult Population**

MSc in Data Science, Universitat Oberta de Catalunya (UOC)

**Joana Llauradó Pont**, Master Student  
**Laia Carreté Muñoz**, Project Supervisor  
Laia Subirats, Coordinator Professor

---

### SECTION 0: GET DATA

Join different dataframes and clean them to build a harmonized dataset for training the models.

---


**INIT WORKING ENVIORNMENT:**

```
# Tiene formato de código
```

 Connect to google drive and import the necessary libraries

In [1]:
# Connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Import libraries
import os
import pandas as pd

#### **LOAD DATA**

Load the different dataframes and take a quick look to get a general idea of the data we have data.

In [3]:
raw_data_symptoms = pd.read_table('/content/drive/MyDrive/TFM 2024/Dades/datos_UOC_sintomas.txt', sep ="\t")
print('SYMPTOMS DATA:', raw_data_symptoms.shape, '\n', raw_data_symptoms.head(), '\n')

SYMPTOMS DATA: (84293, 1) 
                                        ID
UOC00035209  hiperlipidemia, sedentarismo
UOC00035209  hiperlipidemia, sedentarismo
UOC00035209  hiperlipidemia, sedentarismo
UOC00035209  hiperlipidemia, sedentarismo
UOC00035209  hiperlipidemia, sedentarismo 



In [4]:
raw_data_nodiabetes = pd.read_table('/content/drive/MyDrive/TFM 2024/Dades/datos_UOC_nodiabetes.txt', sep =",")
print('NONDIABETES DATA:', raw_data_nodiabetes.shape, '\n', raw_data_nodiabetes.head(), '\n')

NONDIABETES DATA: (31213, 8) 
             ID       Fecha  Edad    Sexo          Prueba  Resultado  \
0  UOC00028388  20/05/2020    52  Hombre      Colesterol      150.0   
1  UOC00028388  26/04/2021    52  Hombre      Colesterol      170.0   
2  UOC00028388  04/04/2018    52  Hombre      Colesterol      181.0   
3  UOC00028388  20/05/2020    52  Hombre  HDL-Colesterol       69.0   
4  UOC00028388  04/04/2018    52  Hombre  HDL-Colesterol       73.0   

   Rango_Inferior  Rango_Superior  
0            70.0             200  
1            70.0             200  
2            70.0             200  
3            40.0              60  
4            40.0              60   



In [5]:
raw_data_diabetes = pd.read_table('/content/drive/MyDrive/TFM 2024/Dades/datos_UOC_diabetes.txt',sep =",")
print('DIABETES DATA:', raw_data_diabetes.shape, '\n', raw_data_diabetes.head())

DIABETES DATA: (53079, 8) 
             ID       Fecha  Edad   Sexo          Prueba  Resultado  \
0  UOC00035209  22/12/2020    54  Mujer      Colesterol      181.0   
1  UOC00035209  22/12/2020    54  Mujer  HDL-Colesterol       65.0   
2  UOC00035209  22/12/2020    54  Mujer  Hb-Glicosilada        7.6   
3  UOC00035209  22/12/2020    54  Mujer   LDL-Calculado       90.0   
4  UOC00035209  22/12/2020    54  Mujer   Trigliceridos      129.0   

   Rango_Inferior  Rango_Superior  
0            70.0             200  
1            40.0              60  
2             3.5               6  
3             0.0             100  
4            35.0             200  


#### **ARRANGE SYMPTOMS DATA**

We observed that the symptoms dataset, one of the three uploaded datasets, has a misplacement in the labels. To address this, we will rename the columns to ensure they are meaningful and consistent. This step is essential for merging the datasets seamlessly later on.


In [6]:
# Drop tue INDEX and rename columns to 'ID' and 'symptoms'
raw_data_symptoms = raw_data_symptoms.reset_index(drop=False)
raw_data_symptoms.columns = ['ID', 'symptoms']
raw_data_symptoms = raw_data_symptoms[raw_data_symptoms['ID'] != 'ID']
print('SYMPTOMS DATA:', raw_data_symptoms.shape, '\n', raw_data_symptoms.head(), '\n')

SYMPTOMS DATA: (84292, 2) 
             ID                      symptoms
0  UOC00035209  hiperlipidemia, sedentarismo
1  UOC00035209  hiperlipidemia, sedentarismo
2  UOC00035209  hiperlipidemia, sedentarismo
3  UOC00035209  hiperlipidemia, sedentarismo
4  UOC00035209  hiperlipidemia, sedentarismo 



### **CHECK OVERLAPPING BETWEEN DATASETS**

To create a final dataset with all the information combined, we first need to check how many patients are in each dataset and identify any overlaps between them. This step also helps us understand the structure of the data we are working with.


In [7]:
# Dimensions of each df + unique IDs
print('N Unique IDs symptoms:', len(raw_data_symptoms['ID'].unique())) #600
print('N Unique IDs non-diabetic:', len(raw_data_nodiabetes['ID'].unique())) #3000
print('N Unique IDs diabetic:', len(raw_data_diabetes['ID'].unique())) #3000

# Check overlap
ids_nondiabetic_symptoms = set(raw_data_nodiabetes['ID']).intersection(raw_data_symptoms['ID'])
ids_diabetic_symptoms = set(raw_data_diabetes['ID']).intersection(raw_data_symptoms['ID'])

print('Overlap non-diabetic and symptoms:', len(ids_nondiabetic_symptoms))
print('Overlap diabetic and symptoms:', len(ids_diabetic_symptoms))

# Find IDs not overlapping
ids_not_overlapping = set(raw_data_nodiabetes['ID']).difference(raw_data_symptoms['ID']).union(set(raw_data_diabetes['ID']).difference(raw_data_symptoms['ID']))
print('IDs not overlapping:', len(ids_not_overlapping))


N Unique IDs symptoms: 6000
N Unique IDs non-diabetic: 3000
N Unique IDs diabetic: 3000
Overlap non-diabetic and symptoms: 3000
Overlap diabetic and symptoms: 3000
IDs not overlapping: 0


We observe that the total dataset includes 6,000 participants, with 3,000 diagnosed with diabetes and 3,000 without diabetes. Additionally, for all participants with symptoms data, there is corresponding information on whether they are diagnosed with or without diabetes. This suggests that we have "complete cases" in terms of participant IDs, meaning all IDs are accounted for across datasets. However, there may still be variability in the number of tests or timepoints available for each participant.


### **MERGE DATA**

To merge the three datasets, we will first add an additional column to the raw data for diabetic and non-diabetic participants called `diagnostic`. This will allow us to track whether each participant belongs to the diabetic or non-diabetic group. After this step, we will combine the datasets, merging the symptoms data with the test results, ensuring all relevant information is unified for each participant.



In [8]:
# Add 'diagnostic' column to distinguish between diabetic and non-diabetic
raw_data_diabetes['diagnostic'] = 'diabetic'
raw_data_nodiabetes['diagnostic'] = 'non_diabetic'

# Combine (row bind) the two DataFrames
combined_data = pd.concat([raw_data_diabetes, raw_data_nodiabetes], ignore_index=True)

# Check the combined data
print(f'Total rows in combined data: {combined_data.shape[0]}')
print(combined_data.head())


Total rows in combined data: 84292
            ID       Fecha  Edad   Sexo          Prueba  Resultado  \
0  UOC00035209  22/12/2020    54  Mujer      Colesterol      181.0   
1  UOC00035209  22/12/2020    54  Mujer  HDL-Colesterol       65.0   
2  UOC00035209  22/12/2020    54  Mujer  Hb-Glicosilada        7.6   
3  UOC00035209  22/12/2020    54  Mujer   LDL-Calculado       90.0   
4  UOC00035209  22/12/2020    54  Mujer   Trigliceridos      129.0   

   Rango_Inferior  Rango_Superior diagnostic  
0            70.0             200   diabetic  
1            40.0              60   diabetic  
2             3.5               6   diabetic  
3             0.0             100   diabetic  
4            35.0             200   diabetic  


In [9]:
# Merge the combined data with symptoms data based on 'ID'
final_merged_data = pd.merge(raw_data_symptoms, combined_data, on='ID', how='left')

# Check the result
print('Total rows in merged data:', final_merged_data.shape[0])
print(combined_data.head())


Total rows in merged data: 1661516
            ID       Fecha  Edad   Sexo          Prueba  Resultado  \
0  UOC00035209  22/12/2020    54  Mujer      Colesterol      181.0   
1  UOC00035209  22/12/2020    54  Mujer  HDL-Colesterol       65.0   
2  UOC00035209  22/12/2020    54  Mujer  Hb-Glicosilada        7.6   
3  UOC00035209  22/12/2020    54  Mujer   LDL-Calculado       90.0   
4  UOC00035209  22/12/2020    54  Mujer   Trigliceridos      129.0   

   Rango_Inferior  Rango_Superior diagnostic  
0            70.0             200   diabetic  
1            40.0              60   diabetic  
2             3.5               6   diabetic  
3             0.0             100   diabetic  
4            35.0             200   diabetic  


### **DATA TYPES CONVERSION**

We need to ensure that the data is in the correct format. To do this, we will check the following:

* **Transform Date Format**: Ensure all date variables are in the correct and consistent format.  
* **Numeric Ranges**: Verify that numeric values fall within acceptable ranges (e.g., age not exceeding 150 years, or HDL cholesterol values within a reasonable threshold).  
* **Missing Values**: Check for any patients missing critical information.  
* **Syntax Formatting**: Remove unnecessary white spaces and standardize text formatting.  
* **Categorical Data**: Verify that categories in the categorical variables make sense and are not duplicated.  
* **Dummy Variables**: Transform categorical variables into dummy variables for modeling purposes.

**References:**  
- [10 Useful Python One-Liners for Data Cleaning](https://www.kdnuggets.com/10-useful-python-one-liners-for-data-cleaning)  
- [Data Cleaning Checklist for Python and Pandas](https://medium.com/@mfflavell/data-cleaning-checklist-for-python-and-pandas-fbe77d11e9d2)



In [10]:
# Convert date columns to datetime with day-first format
if 'Fecha' in final_merged_data.columns:
    final_merged_data['Fecha'] = pd.to_datetime(final_merged_data['Fecha'], dayfirst=True, errors='coerce')

# Convert numeric columns to appropriate types
numeric_columns = ['Edad', 'Resultado', 'Rango_Inferior', 'Rango_Superior']
for col in numeric_columns:
    if col in final_merged_data.columns:
        final_merged_data[col] = pd.to_numeric(final_merged_data[col], errors='coerce')
final_merged_data.head()

,ID,symptoms,Fecha,Edad,Sexo,Prueba,Resultado,Rango_Inferior,Rango_Superior,diagnostic
0,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,Colesterol,181.0,70.0,200,diabetic
1,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,HDL-Colesterol,65.0,40.0,60,diabetic
2,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,Hb-Glicosilada,7.6,3.5,6,diabetic
3,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,LDL-Calculado,90.0,0.0,100,diabetic
4,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,Trigliceridos,129.0,35.0,200,diabetic


In [11]:
# Check - Validate numeric ranges
for prueba, group in final_merged_data.groupby('Prueba'):
    min_range = group['Rango_Inferior'].min()
    max_range = group['Rango_Superior'].max()
    print(f'For {prueba}: Minimum = {min_range}, Maximum = {max_range}')
# Validate numeric ranges for each 'Prueba' grouped by 'diagnostic'
print('\n')
for (prueba, diagnostic), group in final_merged_data.groupby(['Prueba', 'diagnostic']):
    min_range = group['Rango_Inferior'].min()
    max_range = group['Rango_Superior'].max()
    print(f'For {prueba} ({diagnostic}): Minimum = {min_range}, Maximum = {max_range}')

For Colesterol: Minimum = 70.0, Maximum = 218
For HDL-Colesterol: Minimum = 40.0, Maximum = 60
For Hb-Glicosilada: Minimum = 3.5, Maximum = 6
For LDL-Calculado: Minimum = 0.0, Maximum = 218
For Trigliceridos: Minimum = 30.0, Maximum = 200


For Colesterol (diabetic): Minimum = 70.0, Maximum = 218
For Colesterol (non_diabetic): Minimum = 70.0, Maximum = 218
For HDL-Colesterol (diabetic): Minimum = 40.0, Maximum = 60
For HDL-Colesterol (non_diabetic): Minimum = 40.0, Maximum = 60
For Hb-Glicosilada (diabetic): Minimum = 3.5, Maximum = 6
For Hb-Glicosilada (non_diabetic): Minimum = 3.5, Maximum = 6
For LDL-Calculado (diabetic): Minimum = 0.0, Maximum = 218
For LDL-Calculado (non_diabetic): Minimum = 0.0, Maximum = 216
For Trigliceridos (diabetic): Minimum = 30.0, Maximum = 200
For Trigliceridos (non_diabetic): Minimum = 30.0, Maximum = 200


In [12]:
# Trim whitespace in all string columns
for col in final_merged_data.select_dtypes(include='object').columns:
    final_merged_data[col] = final_merged_data[col].str.strip()
final_merged_data.head()

,ID,symptoms,Fecha,Edad,Sexo,Prueba,Resultado,Rango_Inferior,Rango_Superior,diagnostic
0,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,Colesterol,181.0,70.0,200,diabetic
1,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,HDL-Colesterol,65.0,40.0,60,diabetic
2,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,Hb-Glicosilada,7.6,3.5,6,diabetic
3,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,LDL-Calculado,90.0,0.0,100,diabetic
4,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,Trigliceridos,129.0,35.0,200,diabetic


In [13]:
# Check unique values for 'Prueba', 'Sexo', and 'symptoms'
if 'Prueba' in final_merged_data.columns:
    print('Unique values for Prueba:', final_merged_data['Prueba'].unique())

if 'Sexo' in final_merged_data.columns:
    print('Unique values for Sexo:', final_merged_data['Sexo'].unique())

if 'symptoms' in final_merged_data.columns:
    print('Unique values for symptoms:', final_merged_data['symptoms'].unique())

Unique values for Prueba: ['Colesterol' 'HDL-Colesterol' 'Hb-Glicosilada' 'LDL-Calculado'
 'Trigliceridos']
Unique values for Sexo: ['Mujer' 'Hombre']
Unique values for symptoms: ['hiperlipidemia, sedentarismo' 'hipertension'
 'sedentarismo, sed habitual y necesidad de beber' 'Diabetes'
 'sin diagnostico' 'sedentarismo, hipertension, sin diagnostico'
 'sedentarismo, obesidad' 'obesidad, hipertension, fatiga' 'sobrepeso'
 'Diabetes, sobrepeso' 'fatiga, vision borrosa' 'hipotension'
 'sin sintomas' 'anemia, fatiga' 'sedentarismo, sobrepeso' 'sedentarismo']


In [14]:
# Check for missing values in the entire DataFrame
missing_values = final_merged_data.isnull().sum()
print("Missing values in each column:")
print(missing_values)

Missing values in each column:
ID                 0
symptoms           0
Fecha              0
Edad               0
Sexo               0
Prueba             0
Resultado         32
Rango_Inferior     0
Rango_Superior     0
diagnostic         0
dtype: int64


In [15]:
# Convert categorical columns to dummy/indicator variables
categorical_columns = ['Sexo', 'Prueba', 'diagnostic']
dummy_dataframe = pd.get_dummies(final_merged_data[categorical_columns], drop_first=True, dtype=float)

# Merge dummies back to the original DataFrame
final_merged_data = pd.concat([final_merged_data, dummy_dataframe], axis=1)

### **FINAL DATASET**

Save the prepared dataset to use in later stages of the project.



In [17]:
final_merged_data.head()

,ID,symptoms,Fecha,Edad,Sexo,Prueba,Resultado,Rango_Inferior,Rango_Superior,diagnostic,Sexo_Mujer,Prueba_HDL-Colesterol,Prueba_Hb-Glicosilada,Prueba_LDL-Calculado,Prueba_Trigliceridos,diagnostic_non_diabetic
0,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,Colesterol,181.0,70.0,200,diabetic,1.0,0.0,0.0,0.0,0.0,0.0
1,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,HDL-Colesterol,65.0,40.0,60,diabetic,1.0,1.0,0.0,0.0,0.0,0.0
2,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,Hb-Glicosilada,7.6,3.5,6,diabetic,1.0,0.0,1.0,0.0,0.0,0.0
3,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,LDL-Calculado,90.0,0.0,100,diabetic,1.0,0.0,0.0,1.0,0.0,0.0
4,UOC00035209,"hiperlipidemia, sedentarismo",2020-12-22,54,Mujer,Trigliceridos,129.0,35.0,200,diabetic,1.0,0.0,0.0,0.0,1.0,0.0


In [16]:
final_merged_data.to_csv('/content/drive/MyDrive/TFM 2024/Dades/final_merged_data.csv', index=False)